# **Proyecto Instagram**

# Cuenta uno:
### **Email:**

redinstagramapp@gmail.com

redinstagramapp2022


### **Instagram:**

@proyectoredargentina

redinstagram2019


# Cuenta dos:
### **Email:**

redinstagramapp2@gmail.com

redinstagramapp2022


### **Instagram:**

@proyectoredarg

redinstagram2021

## **Facebook:**
redinstagramapp@gmail.com

redinstagramapp2022

In [38]:
# Usuarios a Scrapear
usuarios_interes = [
    
'mauriciomacri', 'cristinafkirchner', 'horaciorodriguezlarreta', 
 'joseluisespert', 'silbatakis', 'kicillofok', 'elisalilitacarrio','sergiomassaok', 
'coqui.capitanich', 'aliciakirchner', 'sergiounac', 'gustavovaldesok'
'juanluismanzur', 'gerardomoralesjujuy', 'gzamorasde', 'jschiaretti'         
'mariuvidal','patobullrich','gugalusto','alferdezok','javiermilei',  
    
]

In [39]:
# Selenium imports
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True
!pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager

#Other imports
import os
import time
from time import sleep
import random
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import Workbook
import re
from statistics import mean

In [40]:
# Usuarios a Scrapear
df = pd.read_excel('./Usuarios/diputados.xlsx')
df.fillna('', inplace=True)
df = df[df['igs']!='']
usuarios_interes = [u.split('/')[3] for u in df['igs'].values]
# usuarios_interes = usuarios_interes[usuarios_interes.index('maggiesolariq'):]

In [41]:
# Credenciales
user = "proyectoredargentina"
password = "redinstagram2019"

In [42]:
class igScraper():
    def __init__(self):
        pass

    def login(self, user, password, navegacion_con_headers = False):

        # Funcion para evadir carteles
        def clickear(texto):
            self.path = f'//*[text() = "{texto}"]'
            self.boton = self.driver.find_element('xpath', self.path) 
            sleep(1)
            self.boton.click()
            sleep(2)

        # instanciamos webdriver (True or False)
        if navegacion_con_headers == True:
            self.driver = webdriver.Chrome(ChromeDriverManager().install())
        else:
            self.driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

        # ir a instagram.com
        self.driver.get('https://instagram.com/')
        sleep(10)

        # poner usuario
        username = self.driver.find_element('name','username')
        sleep(1)
        username.click()
        sleep(1)
        username.send_keys(user)
        sleep(1)

        # poner contraseña
        pw = self.driver.find_element('name','password')
        sleep(2)
        pw.click()
        sleep(2)
        pw.send_keys(password)
        sleep(2)

        # login
        try:
            login_btn = self.driver.find_element('xpath','//div[text() = "Iniciar sesión"]')
        except:
            login_btn = self.driver.find_element('xpath','//div[text() = "Log In"]')
        sleep(2)
        login_btn.click()
        sleep(4)


        # Apretar "ahora no" a los pop ups, hasta que pueda buscarse un usuario
        contador_intentos = 0
        carteles_sorteados = False
        while carteles_sorteados == False and contador_intentos < 200:
            contador_intentos+=1
            try:
                buscador = self.driver.find_element('xpath',"//input[@placeholder='Buscar']")
                carteles_sorteados = True
            except:
                try:
                    buscador = self.driver.find_element('xpath',"//input[@placeholder='Search']")
                    carteles_sorteados = True
                except:
                    try:
                        clickear('Not Now')      
                    except:
                        try:
                            clickear('Ahora no')
                        except:
                            pass

        if contador_intentos == 200:
            print('\n\nEs posible que la cuenta haya sido blockeada.\n\n')

    def descargar(self, usuario, listado_seguidores, nombre):
        print(f'Guardando {nombre} de {usuario}...\n\n')
        os.chdir('Results')
        nombre += '.xlsx'
        sumar = 0
        try:
            wb_obj = openpyxl.load_workbook(nombre)
            sumar += 1
        except:
            Workbook().save(nombre)
            wb_obj = openpyxl.load_workbook(nombre)

        # sheet
        worksheet = wb_obj.worksheets[0]

        # Columna de este usuario
        columna = worksheet.max_column + sumar

        # Nombre columna (usuario)
        worksheet.cell(row = 1, column = columna).value = usuario

        # Debajo, sus seguidos/seguidoress
        for i,s in enumerate(listado_seguidores):
            worksheet.cell(row = i+2, column = columna).value = s

        wb_obj.save(nombre)
        wb_obj.close()
        os.chdir('..')


    # SEGUIDORES
    def get_seguidores(self, usuario, cantidad_usuarios_a_scrapear= float('inf'), download= True):

        print(f'Scrapeando seguidores de {usuario}\n\n')

        url_usuario = 'https://www.instagram.com/'+usuario

        # Entrar al usuario
        self.driver.get(url_usuario)
        sleep(8)

        # Acceder a seguidores
        seguidores_boton = self.driver.find_element('xpath',"//ul[@class='_aa_7']/li[2]//span")
        seguidores_boton.click()
        sleep(3)

        # cantidad de seguidores
        can_seguidores = self.driver.find_element('xpath',"//ul[@class='_aa_7']/li[2]//span").text
        if '.' in can_seguidores:
            cantidad_decimal = len(can_seguidores.split('.')[-1][:-1])
        else:
            cantidad_decimal = 0
        can_seguidores = int(can_seguidores.replace('K','0'*(3-cantidad_decimal)).replace('M','0'*(6-cantidad_decimal)).replace('.','').replace(',',''))
        self.can_seguidores = can_seguidores
        
        # cuanto hay que scrollear
        scrollear_inicial = round(can_seguidores/12)+1

        # instanciamos seguidores_total
        seguidores_total = []
        cantidad_encontrados = []

        # scrollear y apendear
        for i in range(scrollear_inicial):
            i += 1

            #Cada x scrolls, pausa de x minutos
            if i % 10 == 0:
                minutos = random.uniform(3, 7)
                print(f'\n\n\nHaciendo pausa de {round(minutos)} minutos\n\n\n')
                time.sleep(minutos*60)

            # Cada x scrolls, scroll up to confuse bot detectors
            if i % 15 == 0:    
                self.driver.execute_script("""
                    var scrollt = document.querySelector('div[role="dialog"] ._aano')
                    scrollt.scrollTop = scrollt.scrollHeight - (scrollt.scrollHeight / 5)
                    """)

            # Print
            print(f'Scrolleando {i} de {scrollear_inicial}')

            # Scroll down
            self.driver.execute_script("""
                var scrollt = document.querySelector('div[role="dialog"] ._aano')
                scrollt.scrollTop = scrollt.scrollHeight
                """)

            time.sleep(random.uniform(20, 25))

            # Nuevos seguidores
            # Español
            seguidores = self.driver.find_elements('xpath',f"(//div[@class='_aae-']/li)[position()>={len(seguidores_total)}]//*/a")
            seguidores = [u.text for u in seguidores if len(u.text)>0]
            if len(seguidores) == 0:
                #Ingles
                seguidores = self.driver.find_elements('xpath',f"//div[position()>={len(seguidores_total)}]//*/span/a[@role = 'link']")
                seguidores = [elem.get_attribute('href').split('/')[-2] for elem in seguidores]

            # agregamos los nuevos seguidores
            seguidores_total.extend(seguidores)
            print(f'Hasta el momento, se han encontrado {len(seguidores_total)} seguidores de {usuario}\n')

            # Si no encontro nada en las ultimas 3 iters, break
            cantidad_encontrados.append(len(seguidores_total))
            try:
                ultimos = 20
                if len(set(cantidad_encontrados[-ultimos:])) == 1 and len(cantidad_encontrados) > ultimos:
                    print('\n\n\nINTERRUPCION: NO SE ENCUENTRAN NUEVAS CUENTAS\n')
                    break
            except:
                pass

            # break si se alcanzo maximo de usuarios
            if len(seguidores_total) > cantidad_usuarios_a_scrapear:
                break

            time.sleep(random.uniform(20, 25))

        return seguidores_total

    # SEGUIDOS
    def get_seguidos(self, usuario, cantidad_usuarios_a_scrapear= float('inf'), download= True):

        print(f'Scrapeando seguidos de {usuario}\n\n')

        url_usuario = 'https://www.instagram.com/'+usuario

        # Entrar al usuario
        self.driver.get(url_usuario)
        sleep(8)

        # Acceder a seguidos
        seguidos_boton = self.driver.find_element('xpath',"//ul[@class='_aa_7']/li[3]//span")
        seguidos_boton.click()
        sleep(3)

        # cantidad de seguidos
        can_seguidos = self.driver.find_element('xpath',"//ul[@class='_aa_7']/li[3]//span").text
        if '.' in can_seguidos:
            cantidad_decimal = len(can_seguidos.split('.')[-1][:-1])
        else:
            cantidad_decimal = 0
        can_seguidos = int(can_seguidos.replace('K','0'*(3-cantidad_decimal)).replace('M','0'*(6-cantidad_decimal)).replace('.','').replace(',',''))
        self.can_seguidos = can_seguidos
        
        # cuanto hay que scrollear
        scrollear_inicial = round(can_seguidos/12)+1

        # instanciamos seguidos_total
        seguidos_total = []
        cantidad_encontrados = []

        # scrollear y apendear
        for i in range(scrollear_inicial):
            i += 1

            # Cada x scrolls, pausa random
            if i % 10 == 0:
                minutos = random.uniform(3, 7)
                print(f'\n\n\nHaciendo pausa de {round(minutos)} minutos\n\n\n')
                time.sleep(minutos*60)


            # Cada x scrolls, scroll up to confuse bot detectors
            if i % 15 == 0:    
                self.driver.execute_script("""
                    var scrollt = document.querySelector('div[role="dialog"] ._aano')
                    scrollt.scrollTop = scrollt.scrollHeight - (scrollt.scrollHeight / 5)
                    """)

            # Print
            print(f'scrolleando {i} de {scrollear_inicial}')

            # Scroll down
            self.driver.execute_script("""
                var scrollt = document.querySelector('div[role="dialog"] ._aano')
                scrollt.scrollTop = scrollt.scrollHeight
                """)
            time.sleep(random.uniform(20, 25))


            # Nuevos seguidos
            # Español
            seguidos = self.driver.find_elements('xpath',f"(//div[@class='_aae-']/li)[position()>={len(seguidos_total)}]//*/a")
            seguidos = [u.text for u in seguidos if len(u.text)>0]
            if len(seguidos) == 0:
                #Ingles
                seguidos = self.driver.find_elements('xpath',f"//div[position()>={len(seguidos_total)}]//*/span/a[@role = 'link']")
                seguidos = [elem.get_attribute('href').split('/')[-2] for elem in seguidos]

            # agregamos los nuevos seguidos
            seguidos_total.extend(seguidos)
            print(f'Hasta el momento, se han encontrado {len(seguidos_total)} seguidos de {usuario}\n')

            # Si no encontro nada en las ultimas 4 iters, break
            cantidad_encontrados.append(len(seguidos_total))
            try:
                ultimos = 20
                if len(set(cantidad_encontrados[-ultimos:])) == 1 and len(cantidad_encontrados) > ultimos:
                    print('\n\n\nINTERRUPCION: NO SE ENCUENTRAN NUEVAS CUENTAS\n')
                    break
            except:
                pass

            # break si se alcanzo maximo de usuarios
            if len(seguidos_total) > cantidad_usuarios_a_scrapear:
                break

            time.sleep(random.uniform(20, 25))

        return seguidos_total

    def cantidad_usuarios(self, categoria):
        if categoria == 'seguidores':
            return self.can_seguidores
        else:
            return self.can_seguidos
        
    def logout(self):
        # Cerrar webdriver
        self.driver.close()
        print('\nDriver cerrado...\n')

In [ ]:
seguidores_todos = {}
seguidos_todos = {}

tiempos = []

start_total = time.time()

for usuario in usuarios_interes:
    print(f'Scrappeando {usuarios_interes.index(usuario)} de {len(usuarios_interes)}.')
    start = time.time()
    
    porcentaje_encontrado = 0
    tiempo = 60*60
    
    try:
        while porcentaje_encontrado < 0.8:

            # Login
            sc = igScraper()
            sc.login(user, password, navegacion_con_headers = False)

            # Obtener seguidores/seguidos
            seguidos = sc.get_seguidos(usuario)

            porcentaje_encontrado = len(seguidos) / sc.cantidad_usuarios('seguidos')

            sc.logout()

            if porcentaje_encontrado < 0.8:
                tiempo *= 1.1
                print(f'\nHaciendo pausa de {round((tiempo/60/60),2)} horas.'), sleep(tiempo)

        # Guardarlo en dict
        seguidos_todos[usuario] = seguidos

        # Descarga
        sc.descargar(usuario, seguidos, 'seguidos')
        end = time.time()
        t = round((end-start)/60)
        tiempos.append(t)
        print(f'\n\n\n {usuario} tomo {t} minutos, llevamos {sum(tiempos)} y el promedio es de {mean(tiempos)}.\n\n\n')

        # pausa random
        horas = random.uniform(1, 2)
        print(f'\n\n\nHaciendo pausa inter usuario de {round(horas,2)} horas\n\n\n')
        time.sleep(horas*60*60)
        
    except:
        sc.logout()


end_total = time.time()

print(f'\n\n\nTodos los usuarios tomaron {round((end_total-start_total)/60)} minutos\n\n\n')

[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST driver version for 105.0.5195


Scrappeando 0 de 223.
 


[WDM] - Get LATEST driver version for 105.0.5195
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Felipe\.wdm\drivers\chromedriver\win32\105.0.5195.52]


Scrapeando seguidos de bebadesoria


scrolleando 1 de 38
Hasta el momento, se han encontrado 24 seguidos de bebadesoria

scrolleando 2 de 38
Hasta el momento, se han encontrado 36 seguidos de bebadesoria

scrolleando 3 de 38
Hasta el momento, se han encontrado 47 seguidos de bebadesoria

scrolleando 4 de 38
Hasta el momento, se han encontrado 59 seguidos de bebadesoria

scrolleando 5 de 38
Hasta el momento, se han encontrado 71 seguidos de bebadesoria

scrolleando 6 de 38
Hasta el momento, se han encontrado 83 seguidos de bebadesoria

scrolleando 7 de 38
Hasta el momento, se han encontrado 95 seguidos de bebadesoria

scrolleando 8 de 38
Hasta el momento, se han encontrado 103 seguidos de bebadesoria

scrolleando 9 de 38
Hasta el momento, se han encontrado 115 seguidos de bebadesoria




Haciendo pausa de 6 minutos



scrolleando 10 de 38
Hasta el momento, se han encontrado 127 seguidos de bebadesoria

scrolleando 11 de 38
Hasta el momento, se han encontrado 138 seguidos de bebadesoria
